In [1]:
import json

json_path = r"C:\Users\nolan\Desktop\geog4057project\no_tax.json"

with open(json_path, "r") as file:
    data = json.load(file)

print("Total entries:", len(data["data"]))
print("Sample record:", data["data"][0])

Total entries: 496
Sample record: ['row-69eh-dt2h-vwz3', '00000000-0000-0000-A344-B176ECD7FE9B', 0, 1628101573, None, 1628101573, None, '{ }', 'MULTIPOLYGON (((-90.092842237961 29.969376832976, -90.09206523793 29.970255834178, -90.091376237438 29.971000834423, -90.090628237821 29.971830834325, -90.090391236797 29.972100835188, -90.089449236358 29.971550834873, -90.088767236134 29.971163834089, -90.087926235745 29.970586834355, -90.08705723576 29.970059834233, -90.086135234355 29.969567833971, -90.084839233564 29.968734834088, -90.084580233207 29.968568833541, -90.085052233499 29.968045833564, -90.085633233463 29.967421833521, -90.085799233941 29.967243833329, -90.086482234167 29.966464832285, -90.087242234263 29.965596832431, -90.0873702349 29.965447831869, -90.087512234348 29.965550832148, -90.088310235279 29.96610083245, -90.088882235725 29.966494831847, -90.089729235653 29.967051832427, -90.090544236508 29.967635832383, -90.0913742373 29.968210832951, -90.092028237156 29.96865283276

In [3]:
columns = data["meta"]["view"]["columns"]
for i, col in enumerate(columns):
    print(f"{i}: {col['name']}")

0: sid
1: id
2: position
3: created_at
4: created_meta
5: updated_at
6: updated_meta
7: meta
8: the_geom
9: OBJECTID
10: ID
11: Cluster Letter
12: Shape.STArea()
13: Shape.STLength()


In [7]:
import arcpy
import os
import json

json_path = r"C:\Users\nolan\Desktop\geog4057project\no_tax.json"
shapefile_path = r"C:\Users\nolan\Desktop\geog4057project\no_tax.shp"

with open(json_path, "r") as file:
    data = json.load(file)

records = data["data"]
sr = arcpy.SpatialReference(4326)

folder, name = os.path.split(shapefile_path)
arcpy.env.workspace = folder
arcpy.env.overwriteOutput = True
arcpy.management.CreateFeatureclass(folder, name, "POLYGON", spatial_reference=sr)

fields = [("OBJECTID", "LONG"), ("GEOID", "TEXT"), ("Cluster", "TEXT"),
          ("Area", "DOUBLE"), ("Length", "DOUBLE")]
for fname, ftype in fields:
    arcpy.management.AddField(shapefile_path, fname, ftype)

with arcpy.da.InsertCursor(shapefile_path, ["SHAPE@"] + [f[0] for f in fields]) as cursor:
    skipped = 0
    for row in records:
        wkt = row[8].strip()

        if not wkt or wkt in ['{}', 'None', None]:
            skipped += 1
            continue

        try:
            geom = arcpy.FromWKT(wkt, sr)
            attr = [int(row[9]), row[10], row[11], float(row[12]), float(row[13])]
            cursor.insertRow([geom] + attr)
        except:
            skipped += 1
            continue

print(f"{len(records) - skipped} features inserted.")
print(f"{skipped} records skipped due to invalid geometry.")

446 features inserted.
50 records skipped due to invalid geometry.
